In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
import os
from langchain.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.messages import HumanMessage, SystemMessage

from dotenv import load_dotenv

In [2]:
import openai
from getpass import getpass

openai.api_key = getpass("Please provide your OpenAI Key: ")
os.environ["OPENAI_API_KEY"] = openai.api_key

In [3]:
# Установка максимального количества выводимых строк в DataFrame
pd.set_option('display.max_rows', None)

In [4]:
load_dotenv()

True

# Загрузка и чтение файлов в txt

In [253]:


# Получаем текущий рабочий каталог
current_dir = os.getcwd()

# Определяем путь к файлу
file_path = os.path.join(current_dir, "/Users/sergey/Desktop/RAG_Project_FULL/RAG_intelion/Intelion_books/bazaznanii.txt")

# Определяем путь к директории для хранилища
persistent_directory = os.path.join(current_dir, "db", "chroma_db_bazaznanii")


db_dir = os.path.join(current_dir, "db")

# Проверка, существует ли текстовый файл, с которого нужно загружать данные
if not os.path.exists(file_path):
    raise FileNotFoundError(f"Файл {file_path} не найден. Проверьте путь.")
else:
    # Если файл найден, загружаем его содержимое с помощью загрузчика текстовых данных
    loader = TextLoader(file_path)
    documents = loader.load()  # Загружаем документы из текстового файла
    print(f"Загружено {len(documents)} документов.")

# Если документы загружены, используем рекурсивный сплиттер для разбиения
print("\n--- Используем рекурсивное разбиение на чанки ---")
Recursive_text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=150, chunk_overlap=50
)

# Разбиваем документы на чанки
docs = Recursive_text_splitter.split_documents(documents)

print(f"Разбито на {len(docs)} чанков.")

Загружено 1 документов.

--- Используем рекурсивное разбиение на чанки ---
Разбито на 360 чанков.


# Чтение и разбиение файлов на чанки при помощи RecursiveCharacterTextSplitter

In [82]:
# Функция для создания и сохранения векторного хранилища
def create_vector_store(docs, store_name):
    
    # Определение пути к директории, где будет сохранено векторное хранилище
    persistent_directory = os.path.join(db_dir, store_name)
    
    # Проверка, существует ли уже директория для хранения векторного хранилища
    if not os.path.exists(persistent_directory):
        
        # Если директория не существует, выводим сообщение о создании векторного хранилища
        print(f"\n--- Создание векторного хранилища {store_name} ---")
        
        # Создаем векторное хранилище из документов с использованием заданной модели embeddings
        # и сохраняем его в указанную директорию
        db = Chroma.from_documents(
            docs, embeddings, persist_directory=persistent_directory
        )
        
        # Сообщаем, что создание векторного хранилища завершено
        print(f"--- Создание векторного хранилища завершено {store_name} ---")
    
    # Если директория уже существует, выводим сообщение, что хранилище уже создано, и инициализация не требуется
    else:
        print(
            f"Хранилище {store_name} хранилище уже создано, и инициализация не требуется.")



# В качестве эмбеддингов используем text-embedding-3-small от OpenAI

In [254]:
# Загружаем модель эмбеддингов
embeddings = OpenAIEmbeddings(model="text-embedding-3-small",)

In [7]:
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "intfloat/multilingual-e5-base"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

/var/folders/h2/r9wh0x750xq4v5z33ylwrzw40000gn/T/ipykernel_75009/923028654.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/Users/sergey/Desktop/RAG_Project_FULL/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


# Создание векторной базы данных (используем разные эмбеддинги)

In [ ]:
create_vector_store(docs, "chroma_db_intelion_BazaZnanii_OAI_emb")


In [ ]:
create_vector_store(docs, "chroma_db_intelion_BazaZnanii_e5_emb")

# Загрузка векторной базы данных после ее создания или если она была создана ранее

In [255]:
# Укажите путь к вашей базе данных
persistent_directory = "/Users/sergey/Desktop/RAG_Project_FULL/RAG_intelion/db/chroma_db_intelion_BazaZnanii_OAI_emb"

# Проверьте, существует ли база данных
if os.path.exists(os.path.join(persistent_directory, "chroma.sqlite3")):
    print(f"Загружаем векторное хранилище из {persistent_directory}...")

    # Загружаем векторное хранилище
    db = Chroma(
        persist_directory=persistent_directory,  # Указываем путь к существующему хранилищу
        embedding_function=embeddings  # Передаем функцию эмбеддингов (нужна для поиска)
    )

Загружаем векторное хранилище из /Users/sergey/Desktop/RAG_Project_FULL/RAG_intelion/db/chroma_db_intelion_BazaZnanii_OAI_emb...


## Этот код создаёт объект BM25Retriever, который будет использовать алгоритм BM25 (Best Matching 25) для извлечения релевантных документов из набора данных

In [256]:
import rank_bm25
# Создание объекта BM25Retriever на основе переданных документов (docs).
# BM25 — это алгоритм для поиска релевантных документов на основе частоты ключевых слов в документе.
keyword_retriever = BM25Retriever.from_documents(docs)

# Установка количества документов, которые будут возвращаться при каждом поисковом запросе, на 5.
# Это ограничение определяет, что максимум 5 документов будут считаться наиболее релевантными результатами.
keyword_retriever.k = 5

In [257]:
# Преобразование объекта db (который представляет векторную базу данных) в ретривер.
# В данном случае используется тип поиска "similarity", что означает, что поиск будет основан на схожести (например, косинусное расстояние).
retriever_similarity = db.as_retriever(
    
    # Установка типа поиска "similarity". Это указывает на то, что будет использоваться поиск по сходству между векторными представлениями данных.
    search_type="similarity",
    
    # Дополнительные параметры для поиска:
    # "k": 10 — ограничение на количество возвращаемых результатов. Будет возвращено до 10 самых похожих документов.
    search_kwargs={"k": 10},
)

## объект EnsembleRetriever, который объединяет несколько поисковых систем (ретриверов) для извлечения информации.


In [258]:
# Создание объекта EnsembleRetriever, который объединяет несколько ретриверов.
# 'db' и 'keyword_retriever' - это два ретривера, которые будут использованы для поиска.
ensemble_retriever = EnsembleRetriever(retrievers=[retriever_similarity, 
                                                   keyword_retriever],
                                       # Весовые коэффициенты для каждого ретривера.
                                       # Оба ретривера имеют равные веса по 0.5, то есть их вклад в результат одинаков.
                                       weights=[0.6, 0.4])

# Проверяем как работает поиск по базе данных

## OpenAI эмбеддинги

In [ ]:
# Определяем вопрос, на который будем искать ответ
query = "Кто ген дир?"

# Выполняем поиск с использованием EnsembleRetriever
relevant_docs_ensemble = ensemble_retriever.invoke(query)

# Вывод результатов для EnsembleRetriever
print("\n--- Результаты поиска с использованием EnsembleRetriever ---")
for i, doc in enumerate(relevant_docs_ensemble, 1):
    print(f"Документ {i}:\n{doc.page_content}\n")

# e5 эмбеддинги

In [176]:
# Определяем вопрос, на который будем искать ответ
query = "Кто ген дир?"

# Создание объекта EnsembleRetriever, который объединяет несколько ретриверов:
# 'retriever_similarity' и 'keyword_retriever' - два ретривера, участвующие в ансамбле.
ensemble_retriever = EnsembleRetriever(retrievers=[retriever_similarity, 
                                                   keyword_retriever],
                                       # Весовые коэффициенты для каждого ретривера.
                                       # Оба ретривера имеют равные веса по 0.5, что означает, что их вклад в общий результат равнозначен.
                                       weights=[0.6, 0.4])

# Выполняем поиск с использованием EnsembleRetriever
relevant_docs_ensemble = ensemble_retriever.invoke(query)

# Вывод результатов для EnsembleRetriever
print("\n--- Результаты поиска с использованием EnsembleRetriever ---")
for i, doc in enumerate(relevant_docs_ensemble, 1):
    print(f"Документ {i}:\n{doc.page_content}\n")


--- Результаты поиска с использованием EnsembleRetriever ---
Документ 1:
Директор по маркетингу: Денис Губанов

Документ 2:
АР – Александр Александрович Рощин, Операционный директор

Документ 3:
Направление Дизайн
Руководитель Даниил Беленок

Документ 4:
ТА – Тимофей Андреевич Семенов, Генеральный директор

Документ 5:
изменения с руководителем. Если он сам является руководителем подразделения, то обращаться нужно к

Документ 6:
МГ – Максим Геннадьевич Симуткин, Директор по развитию, энергетике и строительству

Документ 7:
отдела, а руководитель отдела — со своим руководителей и HRD.

Документ 8:
Направление SEO
Руководитель Глеб Крячок

Документ 9:
О компании

Документ 10:
Направление Разработка
Руководитель Криштун Роман

Документ 11:
[b]Внутри файл https://intelionmining.bitrix24.ru/knowledge/howintelionworks/filosofiyaintelion_hruw/

Документ 12:
Для строительства нового ЦОДа Intelion обязательно выбираются регионы с низкой стоимостью электроэнергии и свободным объемом электрическ

# 1. Самый простой вариант
## здесь добовляем генерацию ответа LLM

## Просто поиск по векторной базе 

In [177]:
# Определяем вопрос, на который будем искать ответ
query = "Кто ген дир?"

# 1. Поиск по сходству без порога (search_type="similarity")
retriever_similarity = db.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 15},
)

# Выполняем запрос к ансамблю ретриверов для получения релевантных документов
relevant_docs = retriever_similarity.invoke(query)

# Выводим релевантные документы с их содержимым
print("\n--- Релевантные документы ---")
for i, doc in enumerate(relevant_docs, 1):
    # Выводим каждый документ с его содержимым и номером
    print(f"Документ {i}:\n{doc.page_content}\n")

# Объединяем запрос и содержимое релевантных документов в один текст для передачи модели
combined_input = (
    "Это некоторые документы которые могут помочь ответить на вопрос: "
    + query  # Включаем исходный запрос
    + "\n\nРелевантные документы:\n"  # Добавляем заголовок для раздела с документами
    + "\n\n".join([doc.page_content for doc in relevant_docs])  # Добавляем текст всех документов
    + "\n\nПожалуйста, предоставьте ответ, основываясь только на предоставленных документах. Если ответ не найден в документах, ответьте ‘Я не уверен'."  # Просим модель ответить только на основе предоставленных данных
)

# Создаем модель ChatOpenAI для обработки запроса
model = ChatOpenAI(model="gpt-4o-mini")

# Определяем сообщения, которые передаем модели
messages = [
    SystemMessage(content="Ты помощник, который отвечает на вопросы пользователя."),  # Системное сообщение с инструкциями для модели
    HumanMessage(content=combined_input),  # Сообщение пользователя, содержащее запрос и документы
]

# Вызываем модель с подготовленными сообщениями
result = model.invoke(messages)

# Выводим сгенерированный ответ
print("\n--- Сгенерированный ответ ---")
print("Текстовый ответ модели:")
print(result.content)


--- Релевантные документы ---
Документ 1:
Директор по маркетингу: Денис Губанов

Документ 2:
АР – Александр Александрович Рощин, Операционный директор

Документ 3:
Направление Дизайн
Руководитель Даниил Беленок

Документ 4:
ТА – Тимофей Андреевич Семенов, Генеральный директор

Документ 5:
изменения с руководителем. Если он сам является руководителем подразделения, то обращаться нужно к

Документ 6:
МГ – Максим Геннадьевич Симуткин, Директор по развитию, энергетике и строительству

Документ 7:
отдела, а руководитель отдела — со своим руководителей и HRD.

Документ 8:
Направление SEO
Руководитель Глеб Крячок

Документ 9:
Организационные вопросы

Документ 10:
О компании

Документ 11:
Направление Разработка
Руководитель Криштун Роман

Документ 12:
Операционный директор Александр Рощин

Документ 13:
ЮИ – Юлия Игоревна Яровова, Директор по правовым вопросам

Документ 14:
Юридический отдел
Директор по правовым вопросам Юлия Игоревна Яровова

Документ 15:
их руководителю своего отдела, HRD Ан

## Поиск по обьедененным векторной базе данных и поиск по ключевым словам 

In [178]:
# Определяем вопрос, на который будем искать ответ
query = "Кто ген дир?"

# Выполняем запрос к ансамблю ретриверов для получения релевантных документов
relevant_docs = ensemble_retriever.invoke(query)

# Выводим релевантные документы с их содержимым
print("\n--- Релевантные документы ---")
for i, doc in enumerate(relevant_docs, 1):
    # Выводим каждый документ с его содержимым и номером
    print(f"Документ {i}:\n{doc.page_content}\n")

# Объединяем запрос и содержимое релевантных документов в один текст для передачи модели
combined_input = (
    "Это некоторые документы которые могут помочь ответить на вопрос: "
    + query  # Включаем исходный запрос
    + "\n\nРелевантные документы:\n"  # Добавляем заголовок для раздела с документами
    + "\n\n".join([doc.page_content for doc in relevant_docs])  # Добавляем текст всех документов
    + "\n\nПожалуйста, предоставьте ответ, основываясь только на предоставленных документах. Если ответ не найден в документах, ответьте ‘Я не уверен'."  # Просим модель ответить только на основе предоставленных данных
)

# Создаем модель ChatOpenAI для обработки запроса
model = ChatOpenAI(model="gpt-4o-mini")

# Определяем сообщения, которые передаем модели
messages = [
    SystemMessage(content="Ты помощник, который отвечает на вопросы пользователя."),  # Системное сообщение с инструкциями для модели
    HumanMessage(content=combined_input),  # Сообщение пользователя, содержащее запрос и документы
]

# Вызываем модель с подготовленными сообщениями
result = model.invoke(messages)

# Выводим сгенерированный ответ
print("\n--- Сгенерированный ответ ---")
print("Текстовый ответ модели:")
print(result.content)


--- Релевантные документы ---
Документ 1:
Директор по маркетингу: Денис Губанов

Документ 2:
АР – Александр Александрович Рощин, Операционный директор

Документ 3:
Направление Дизайн
Руководитель Даниил Беленок

Документ 4:
ТА – Тимофей Андреевич Семенов, Генеральный директор

Документ 5:
изменения с руководителем. Если он сам является руководителем подразделения, то обращаться нужно к

Документ 6:
МГ – Максим Геннадьевич Симуткин, Директор по развитию, энергетике и строительству

Документ 7:
отдела, а руководитель отдела — со своим руководителей и HRD.

Документ 8:
Направление SEO
Руководитель Глеб Крячок

Документ 9:
О компании

Документ 10:
Направление Разработка
Руководитель Криштун Роман

Документ 11:
[b]Внутри файл https://intelionmining.bitrix24.ru/knowledge/howintelionworks/filosofiyaintelion_hruw/

Документ 12:
Для строительства нового ЦОДа Intelion обязательно выбираются регионы с низкой стоимостью электроэнергии и свободным объемом электрической мощности

Документ 13:
с кру

In [21]:
from langchain.chains import create_history_aware_retriever, create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

 # 2. Вариант с перефразированием и использованием контекста всей беседы

# Вариант с выводом перефразированных вопросов

## Только векторная база данных

In [223]:
# Создаем модель ChatOpenAI с использованием модели GPT-4o
llm = ChatOpenAI(model="gpt-4o-mini")  # Модель GPT-4o для обработки запросов

# Определяем системный промпт для контекстуализации вопросов
contextualize_q_system_prompt = (
    "Учитывая историю чата и последний вопрос пользователя, "
    "который может ссылаться на контекст из истории чата, "
    "сформулируйте самодостаточный вопрос, который можно понять "
    "без использования истории чата. НЕ отвечайте на вопрос, просто "
    "переформулируйте его, если это необходимо, или верните его без изменений."
)

# Создаем шаблон для формирования контекстуализированных вопросов
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Создаем retriever, учитывающий историю чата
history_aware_retriever = create_history_aware_retriever(
    llm, retriever_similarity, contextualize_q_prompt
)

# Определяем системный промпт для ответа на вопросы
qa_system_prompt = (
    "Вы помощник для задач по ответам на вопросы из справочника конструктора машиностроителя. "
    "Используйте следующие части полученного контекста, чтобы ответить на вопрос. "
    "Используйте только контекст, полученный из базы данных, чтобы ответить на вопрос. "
    "Если контекст не найден или не подходит для ответа, скажите, что вы не знаете ответа. "
    "Используйте максимум три предложения. Ответ должен быть кратким."
    "{context}"
)

# Создаем шаблон для формирования ответов на вопросы
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Создаем цепочку для объединения документов для ответов на вопросы
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Создаем retrieval-цепочку, которая объединяет retriever с историей и цепочку для ответов на вопросы
simple_vector_store_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

# Функция для моделирования непрерывного чата с выводом переформулированного вопроса и релевантных документов из базы данных
def continual_chat():
    print("Начните общение с ИИ! Введите ‘выход’, чтобы завершить разговор.")  # Начало общения с ИИ
    chat_history = []  # Хранение истории чата
    while True:
        query = input("ВЫ: ")  # Получение запроса от пользователя
        if query.lower() == "выход":  # Если пользователь хочет завершить чат
            break

        # Переформулировка вопроса через вызов LLM с контекстом
        reformulated_prompt = contextualize_q_prompt.format_messages(
            chat_history=chat_history, input=query
        )
        reformulated_query_result = llm(reformulated_prompt)  # Получаем объект AIMessage
        reformulated_query = reformulated_query_result.content  # Доступ к содержимому через атрибут content

        # Получение релевантных документов напрямую из базы данных
        relevant_docs = ensemble_retriever.get_relevant_documents(reformulated_query)

        # Печать исходного вопроса, переформулированного вопроса и релевантных документов с метриками
        print(f"Вы: {query}")  # Печать исходного вопроса пользователя
        print(f"Переформулированный вопрос: {reformulated_query}")  # Вывод переформулированного вопроса

        # Обработка запроса пользователя через цепочку retrieval для получения финального ответа
        retrieval_result = simple_vector_store_chain.invoke({"input": reformulated_query, "chat_history": chat_history})  # Запрос к цепочке

        # Печать ответа ИИ
        print(f"AI: {retrieval_result['answer']}")  # Ответ ИИ

        # Обновление истории чата
        chat_history.append(HumanMessage(content=query))  # Добавление сообщения пользователя в историю
        chat_history.append(SystemMessage(content=retrieval_result["answer"]))  # Добавление ответа ИИ в историю

# Main function to start the continual chat
if __name__ == "__main__":
    continual_chat()

Начните общение с ИИ! Введите ‘выход’, чтобы завершить разговор.
Вы: Кто ген дир?
Переформулированный вопрос: Кто является генеральным директором?
AI: Генеральным директором является Тимофей Андреевич Семенов.


## Поиск по векторной базе и ключевым словам 

In [224]:
# Создаем модель ChatOpenAI с использованием модели GPT-4o
llm = ChatOpenAI(model="gpt-4o-mini")  # Модель GPT-4o для обработки запросов

# Определяем системный промпт для контекстуализации вопросов
contextualize_q_system_prompt = (
    "Учитывая историю чата и последний вопрос пользователя, "
    "который может ссылаться на контекст из истории чата, "
    "сформулируйте самодостаточный вопрос, который можно понять "
    "без использования истории чата. НЕ отвечайте на вопрос, просто "
    "переформулируйте его, если это необходимо, или верните его без изменений."
)

# Создаем шаблон для формирования контекстуализированных вопросов
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),  # Системное сообщение с инструкцией для ИИ
        MessagesPlaceholder("chat_history"),  # Место для истории чата
        ("human", "{input}"),  # Ввод пользователя, который нужно переформулировать
    ]
)

# Создаем retriever, учитывающий историю чата
history_aware_retriever = create_history_aware_retriever(
    llm, ensemble_retriever, contextualize_q_prompt  # Используем созданный шаблон и retriever для поиска с историей
)

# Определяем системный промпт для ответа на вопросы
qa_system_prompt = (
    "Вы помощник для задач по ответам на вопросы из справочника конструктора машиностроителя. "
    "Используйте следующие части полученного контекста, чтобы ответить на вопрос. "
    "Используйте только контекст, полученный из базы данных, чтобы ответить на вопрос. "
    "Если контекст не найден или не подходит для ответа, скажите, что вы не знаете ответа. "
    "Используйте максимум три предложения. Ответ должен быть кратким."
    "{context}"
)

# Создаем шаблон для формирования ответов на вопросы
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),  # Системное сообщение с инструкцией для ИИ
        MessagesPlaceholder("chat_history"),  # Место для истории чата
        ("human", "{input}"),  # Ввод пользователя, на который нужно ответить
    ]
)

# Создаем цепочку для объединения документов для ответов на вопросы
# `create_stuff_documents_chain` передает весь найденный контекст в LLM для обработки
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Создаем retrieval-цепочку, которая объединяет retriever с историей и цепочку для ответов на вопросы
simple_vector_store_and_BM25 = create_retrieval_chain(history_aware_retriever, question_answer_chain)

# Функция для моделирования непрерывного чата с выводом переформулированного вопроса и релевантных документов из базы данных
def continual_chat():
    print("Начните общение с ИИ! Введите ‘выход’, чтобы завершить разговор.")  # Начало общения с ИИ
    chat_history = []  # Хранение истории чата
    while True:
        query = input("ВЫ: ")  # Получение запроса от пользователя
        if query.lower() == "выход":  # Если пользователь хочет завершить чат
            break

        # Переформулировка вопроса через вызов LLM с контекстом
        reformulated_prompt = contextualize_q_prompt.format_messages(
            chat_history=chat_history, input=query
        )
        reformulated_query_result = llm(reformulated_prompt)  # Получаем объект AIMessage
        reformulated_query = reformulated_query_result.content  # Доступ к содержимому через атрибут content

        # Получение релевантных документов напрямую из базы данных
        relevant_docs = ensemble_retriever.get_relevant_documents(reformulated_query)

        # Печать исходного вопроса, переформулированного вопроса и релевантных документов с метриками
        print(f"Вы: {query}")  # Печать исходного вопроса пользователя
        print(f"Переформулированный вопрос: {reformulated_query}")  # Вывод переформулированного вопроса

        # Обработка запроса пользователя через цепочку retrieval для получения финального ответа
        retrieval_result = simple_vector_store_and_BM25.invoke({"input": reformulated_query, "chat_history": chat_history})  # Запрос к цепочке

        # Печать ответа ИИ
        print(f"AI: {retrieval_result['answer']}")  # Ответ ИИ

        # Обновление истории чата
        chat_history.append(HumanMessage(content=query))  # Добавление сообщения пользователя в историю
        chat_history.append(SystemMessage(content=retrieval_result["answer"]))  # Добавление ответа ИИ в историю

# Main function to start the continual chat
if __name__ == "__main__":
    continual_chat()

Начните общение с ИИ! Введите ‘выход’, чтобы завершить разговор.
Вы: Кто ген дир?
Переформулированный вопрос: Кто является генеральным директором?
AI: Генеральным директором является Тимофей Андреевич Семенов.


## То же самое что и выше только с использованием переранжирования

In [225]:
from langchain_community.document_transformers import LongContextReorder

# Создаем модель ChatOpenAI с использованием модели GPT-4o
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)  # Модель GPT-4o для обработки запросов

# Определяем системный промпт для контекстуализации вопросов
contextualize_q_system_prompt = (
    "Учитывая историю чата и последний вопрос пользователя, "
    "который может ссылаться на контекст из истории чата, "
    "сформулируйте самодостаточный вопрос, который можно понять "
    "без использования истории чата. НЕ отвечайте на вопрос, просто "
    "переформулируйте его, если это необходимо, или верните его без изменений."
)

# Создаем шаблон для формирования контекстуализированных вопросов
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Создаем retriever, учитывающий историю чата
history_aware_retriever = create_history_aware_retriever(
    llm, ensemble_retriever, contextualize_q_prompt
)

# Определяем системный промпт для ответа на вопросы
qa_system_prompt = (
    "Вы помощник для задач по ответам на вопросы о компании Intelion. "
    "Используйте следующие части полученного контекста, чтобы ответить на вопрос. "
    "Если вы не знаете ответа, просто скажите, что вы не знаете. "
    "Используйте максимум три предложения и дайте краткий ответ."
    "\n\n{context}"
)

# Создаем шаблон для формирования ответов на вопросы
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Создаем цепочку для объединения документов для ответов на вопросы
question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

# Создаем объект LongContextReorder для переранжирования длинных контекстов
long_context_reorder = LongContextReorder()

# Создаем базовую retrieval-цепочку без переранжирования
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

# Создаем цепочку с переранжированием для документов — назовем её simple_reranked
def simple_reranked_chain(input_query, chat_history):
    # Получение результата от базовой retrieval-цепочки
    result = rag_chain.invoke({"input": input_query, "chat_history": chat_history})

    # Применение LongContextReorder для переранжирования длинных документов
    reordered_docs = long_context_reorder.transform_documents(result["context"])

    # Возвращаем переранжированные документы и ответ ИИ
    return {"answer": result["answer"], "reordered_docs": reordered_docs}

# Функция для моделирования непрерывного чата с использованием simple_reranked_chain
def continual_chat():
    print("Начните общение с ИИ! Введите ‘выход’, чтобы завершить разговор.")
    chat_history = []  # Хранение истории чата
    while True:
        query = input("ВЫ: ")  # Получение запроса от пользователя
        if query.lower() == "выход":  # Если пользователь хочет завершить чат
            break

        # Использование цепочки с переранжированием
        result = 
        (query, chat_history)
        
        # Печать вопроса пользователя и ответа ИИ
        print(f"Вы: {query}")
        print(f"AI: {result['answer']}")  # Ответ ИИ
        # print(f"Переранжированные документы: {result['reordered_docs']}")  # Переранжированные документы
        
        # Обновление истории чата
        chat_history.append(HumanMessage(content=query))  # Добавление сообщения пользователя в историю
        chat_history.append(SystemMessage(content=result["answer"]))  # Добавление ответа ИИ в историю

# Main function to start the continual chat
if __name__ == "__main__":
    continual_chat()

Начните общение с ИИ! Введите ‘выход’, чтобы завершить разговор.
Вы: Кто ген дир?
AI: Генеральный директор компании Intelion — Тимофей Андреевич Семенов.


LongContextReorder выполняет переранжирование документов, чтобы минимизировать эффект “затерянности посередине”. Описание работы:

	1.	Исходное ранжирование:
	•	Документы от retriever'а поступают в порядке убывания релевантности (наиболее релевантные документы в начале списка).
	2.	Алгоритм изменения порядка:
	•	LongContextReorder перераспределяет документы так, чтобы наиболее релевантные документы оказались в начале и в конце списка, а менее релевантные документы — в середине.

	Конкретные шаги:
	•	Берется первый и последний документ из исходного списка (самые релевантные) и размещаются на крайних позициях (первый и последний).
	•	Следующий по релевантности документ размещается во вторую позицию, предпоследний — на вторую с конца, и так далее.
	•	Таким образом, релевантные документы занимают ключевые позиции (начало и конец), что помогает модели сфокусироваться на важных данных в начале и конце контекста, а не пропускать 	их 	в середине.
	3.	Результат:
	•	Выводится упорядоченный список документов, в котором релевантные документы на краях контекста, что может повысить качество ответов языковой модели.

# 3. Поиск документов с использованием RAG-Fusion и RRF для поиска и ранжирования документов:
## После переформулировки вопрос передается в генерацию нескольких запросов, и система выполняет поиск документов для каждого из них.

# Этот вариант работает медленнее и не сказал бы что ответы лучше

In [227]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain.load import dumps, loads
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

# Создаем модель ChatOpenAI с использованием модели GPT-4o
llm = ChatOpenAI(model="gpt-4o-mini")  # Модель GPT-4o для обработки запросов

# Определяем системный промпт для контекстуализации вопросов
contextualize_q_system_prompt = (
    "Учитывая историю чата и последний вопрос пользователя, "
    "который может ссылаться на контекст из истории чата, "
    "сформулируйте самодостаточный вопрос, который можно понять "
    "без использования истории чата. НЕ отвечайте на вопрос, просто "
    "переформулируйте его, если это необходимо, или верните его без изменений."
)

# Создаем шаблон для формирования контекстуализированных вопросов
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Шаблон для генерации нескольких запросов
template = """Вы являетесь полезным помощником и знаете все о компании Intelion, который генерирует несколько поисковых запросов на основе одного входного запроса. \n
Сгенерируйте несколько поисковых запросов, связанных с: {question} \n
Output (4 queries):"""

prompt_rag_fusion = ChatPromptTemplate.from_template(template)

# Функция для генерации нескольких запросов
generate_queries = (
    prompt_rag_fusion 
    | ChatOpenAI(temperature=0) 
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

# Функция для Reciprocal Rank Fusion (RRF)
def reciprocal_rank_fusion(results: list[list], k=60):
    """Функция RRF для слияния результатов поиска из нескольких списков."""
    fused_scores = {}
    for docs in results:
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            fused_scores[doc_str] += 1 / (rank + k)
    
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    
    return reranked_results

# Создаем retrieval-цепочку с использованием ensemble_retriever
def create_rag_fusion_retrieval_chain(ensemble_retriever, question):
    # Генерируем несколько запросов
    generated_queries = generate_queries.invoke({"question": question})
    
    # Выполняем поиск по базе данных для каждого запроса с использованием ensemble_retriever
    # ensemble_retriever предполагает, что используется несколько различных поисковых движков
    search_results = []
    for query in generated_queries:
        # Для каждого запроса вызываем ensemble_retriever и собираем результаты
        retrieved_documents = ensemble_retriever.get_relevant_documents(query)
        search_results.append(retrieved_documents)
    
    # Применяем RRF для слияния результатов
    reranked_docs = reciprocal_rank_fusion(search_results)
    
    # Извлекаем только документы, без их рангов
    return [doc for doc, score in reranked_docs]

# Создаем шаблон для ответа на вопросы на основе контекста
qa_system_prompt = (
    "Вы помощник для задач по ответам на вопросы и знаете все о компании Intelion. Используйте "
    "следующие части полученного контекста, чтобы ответить на "
    "вопрос. Если вы не знаете ответа, просто скажите, что вы "
    "не знаете. Используйте максимум три предложения и дайте "
    "краткий ответ.\n\n"
    "{context}"
)

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

# Функция для обработки запроса с контекстуализацией и RAG-Fusion
def process_query(query, chat_history):
    # Шаг 1: Переформулируем вопрос с учетом истории чата
    reformulated_prompt = contextualize_q_prompt.format_messages(
        chat_history=chat_history, input=query
    )
    reformulated_query_result = llm(reformulated_prompt)  # Получаем объект AIMessage
    reformulated_query = reformulated_query_result.content  # Доступ к содержимому
    
    # Шаг 2: Получаем документы с использованием RAG-Fusion и RRF через ensemble_retriever
    docs = create_rag_fusion_retrieval_chain(ensemble_retriever, reformulated_query)
    
    # Шаг 3: Генерируем ответ на основе контекста
    result = question_answer_chain.invoke({"input": query, "context": docs, "chat_history": chat_history})
    
    return result

# Функция для моделирования непрерывного чата
def continual_chat():
    print("Начните общение с ИИ! Введите ‘выход’, чтобы завершить разговор.")
    chat_history = []  # Хранение истории чата
    while True:
        query = input("ВЫ: ")  # Получение запроса от пользователя
        if query.lower() == "выход":  # Если пользователь хочет завершить чат
            break
        
        # Обрабатываем запрос пользователя через систему RAG-Fusion с RRF
        result = process_query(query, chat_history)
        
        # Печать вопроса пользователя и ответа ИИ
        print(f"Вы: {query}")
        print(f"AI: {result}")  # Выводим результат, так как это строка
        
        # Обновление истории чата
        chat_history.append(HumanMessage(content=query))
        chat_history.append(SystemMessage(content=result))

# Запуск непрерывного чата
if __name__ == "__main__":
    continual_chat()

Начните общение с ИИ! Введите ‘выход’, чтобы завершить разговор.
Вы: кто ген дир?
AI: Генеральным директором компании Intelion является Екатерина Евгеньевна Семыкина.


# Оцениваем при помощи RAGAS работу наших RAG

## Создаем датасет из нашей базы данных с 
 ### question — вопрос пользователя, который он подает в RAG;
 ### contexts — контексты, использовавшиеся при ответе на вопрос;
 ### ground_truth — верный ответ на вопрос пользователя.

In [23]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI
import pandas as pd
from tqdm import tqdm

In [24]:
# Шаг 1: Подготовка схемы и парсера для генерации вопросов
question_schema = ResponseSchema(
    name="question",
    description="вопрос о контексте."
)

question_response_schemas = [question_schema]
question_output_parser = StructuredOutputParser.from_response_schemas(question_response_schemas)

# GPT-3.5 модель с минимальными параметрами
question_generation_llm = ChatOpenAI(
    model_name="gpt-3.5-turbo"  # Обновленный параметр 'model_name'
)

In [25]:
# Шаблон для генерации вопросов
qa_template = """\
Вы помощник для задач по ответам на вопросы о компании Intelion. Для каждого контекста создайте конкретный вопрос, который относится именно к предоставленному контексту. Избегайте общих или слишком обобщённых вопросов.

question: вопрос, относящийся к контексту.

Отформатируйте вывод в формате JSON с ключами:
question

context: {context}
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)

# Шаг 2: Генерация вопросов по первым 30 чанкам контекста
qac_triples = []

# Ограничим до первых 30 чанков
for text in tqdm(docs[:30]):
    # Форматирование сообщений для модели
    messages = prompt_template.format_messages(
        context=text.page_content
    )
    
    # Вызов модели для генерации вопроса
    response = question_generation_llm(messages)
    try:
        output_dict = question_output_parser.parse(response.content)
    except Exception as e:
        continue
    # Добавляем в список: вопрос и контекст
    output_dict["context"] = text.page_content
    qac_triples.append(output_dict)

# Шаг 3: Подготовка схемы и парсера для генерации ответов
answer_schema = ResponseSchema(
    name="answer",
    description="ответ на вопрос"
)

100%|██████████| 30/30 [00:29<00:00,  1.02it/s]


In [26]:
answer_response_schemas = [answer_schema]
answer_output_parser = StructuredOutputParser.from_response_schemas(answer_response_schemas)

# Шаблон для генерации ответов
qa_template = """\
Вы помощник для задач по ответам на вопросы о компании Intelion. Для каждого вопроса и контекста создайте конкретный и точный ответ.

answer: ответ на вопрос, относящийся к контексту.

Отформатируйте вывод в формате JSON с ключами:
answer

question: {question}
context: {context}
"""

prompt_template = ChatPromptTemplate.from_template(template=qa_template)

# gpt-4o-mini модель для ответов
answer_generation_llm = ChatOpenAI(model="gpt-4o-mini")

# Шаг 4: Генерация ответов по каждому вопросу и контексту для первых 30 чанков
for triple in tqdm(qac_triples):
    messages = prompt_template.format_messages(
        context=triple["context"],
        question=triple["question"]
    )
    
    # Вызов модели для генерации ответа
    response = answer_generation_llm(messages)
    try:
        output_dict = answer_output_parser.parse(response.content)
    except Exception as e:
        continue
    # Добавляем ответ в тройку данных
    triple["answer"] = output_dict["answer"]

100%|██████████| 30/30 [00:43<00:00,  1.45s/it]


In [27]:
# Шаг 5: Сохранение данных в pandas DataFrame
ground_truth_qac_set = pd.DataFrame(qac_triples)

# Сохранение контекста в строковом виде
ground_truth_qac_set["context"] = ground_truth_qac_set["context"].map(lambda x: str(x))

# Переименуем колонку с ответами для ясности
ground_truth_qac_set = ground_truth_qac_set.rename(columns={"answer": "ground_truth"})

# Выводим DataFrame
ground_truth_qac_set.head()

,question,context,ground_truth
0,Какая область деятельности компании Intelion?,﻿Общая информация о компании Intelion\nДобрый ...,Компания Intelion занимается разработкой и вне...
1,Какая основная информация содержится в данном ...,"В этом документе собрана основная информация, ...",В данном документе содержится основная информа...
2,Какие основные сферы деятельности компании Int...,для первого знакомства с компанией. Другие воп...,Основные сферы деятельности компании Intelion ...
3,Какую анкету следует использовать для добавлен...,Если вы хотите добавить какую-то информацию в ...,Воспользуйтесь предоставленной анкетой для доб...
4,Какие технологии в области искусственного инте...,"данных, воспользуйтесь этой анкетой: https://...",Компания Intelion использует различные техноло...


In [123]:
ground_truth_qac_set['question'][1]

'Какая основная информация содержится в данном документе о компании Intelion?'

In [124]:
ground_truth_qac_set['context'][1]

'В этом документе собрана основная информация, необходимая для первого знакомства с компанией.'

In [125]:
ground_truth_qac_set['ground_truth'][1]

'В данном документе содержится основная информация о компании Intelion, которая может включать ее историю, миссию, основные направления деятельности, продукты и услуги, а также контактные данные для связи.'

In [126]:
from datasets import Dataset
eval_dataset = Dataset.from_pandas(ground_truth_qac_set)

In [184]:
eval_dataset

Dataset({
    features: ['question', 'context', 'ground_truth'],
    num_rows: 30
})

In [128]:
from ragas.metrics import (
    answer_relevancy,
    faithfulness,
    context_recall,
    context_precision,
    answer_correctness,
    answer_similarity
)

In [129]:
from ragas import evaluate
from tqdm import tqdm
import pandas as pd
from datasets import Dataset

## Оценка простого RAG с использованием только поиска по векторной базе данных

In [229]:
import pandas as pd
from tqdm import tqdm

# Оценка RAG-набора данных с использованием RAGAS
def evaluate_ragas_dataset(ragas_dataset):
    result = evaluate(
        ragas_dataset,
        metrics=[
            context_precision,
            faithfulness,
            answer_relevancy,
            context_recall,
            answer_correctness,
            answer_similarity
        ],
    )
    return result

# Функция для создания RAG-набора данных с использованием цепочки simple_vector_store_chain
def create_ragas_dataset_with_simple_chain(simple_chain, eval_dataset):
    rag_dataset = []
    
    # Преобразуем Dataset в pandas DataFrame для удобной работы с данными
    eval_df = pd.DataFrame(eval_dataset)
    
    for _, row in tqdm(eval_df.iterrows()):  # Итерируем через строки DataFrame
        try:
            # Вызов цепочки через метод invoke
            result = simple_chain.invoke({"input": row["question"], "chat_history": []})
            
            # Проверка на наличие ключей
            if not result or "answer" not in result:
                print(f"Ошибка: отсутствует ключ 'answer' для вопроса: {row['question']}")
                continue
            
            # Получаем релевантные документы (если есть)
            relevant_docs = result.get("relevant_docs", [])
            contexts = [doc.page_content for doc in relevant_docs]

            # Добавление данных в RAG-набор
            rag_dataset.append(
                {
                    "question": row["question"],
                    "answer": result["answer"],  # Ответ от цепочки
                    "contexts": contexts,  # Оставляем контексты в виде списка
                    "ground_truths": row["ground_truth"],  # Эталонные ответы
                    "reference": row["ground_truth"]  # Эталонный ответ для оценки
                }
            )
        except Exception as e:
            print(f"Ошибка обработки вопроса '{row['question']}': {e}")
    
    # Преобразуем итоговый набор в pandas DataFrame
    if rag_dataset:  # Проверяем, что набор данных не пустой
        rag_df = pd.DataFrame(rag_dataset)
        rag_eval_dataset = Dataset.from_pandas(rag_df)
        return rag_eval_dataset
    else:
        print("Ошибка: набор данных пустой.")
        return None

# Создание RAG-набора данных для оценки с использованием цепочки simple_vector_store_chain (первые 10 строк)
basic_qa_ragas_dataset_simple = create_ragas_dataset_with_simple_chain(simple_vector_store_chain, eval_dataset[:10])

# Проверка, что создание RAG-набора данных завершилось успешно
if basic_qa_ragas_dataset_simple is not None:
    # Сохранение RAG-набора данных в CSV
    basic_qa_ragas_dataset_simple.to_csv("basic_qa_ragas_dataset_simple.csv")

    # Оценка RAG-набора данных
    basic_qa_vector_store = evaluate_ragas_dataset(basic_qa_ragas_dataset_simple)

    # Вывод результата оценки
    print(basic_qa_vector_store)
else:
    print("Ошибка создания RAG-набора данных. Оценка не будет выполнена.")

10it [00:12,  1.24s/it]
Evaluating: 100%|██████████| 60/60 [00:21<00:00,  2.79it/s]


{'context_precision': 0.0000, 'faithfulness': 0.3000, 'answer_relevancy': 0.7023, 'context_recall': 0.0000, 'answer_correctness': 0.4054, 'answer_similarity': 0.9098}


## Оценка простого RAG с использованием поиска по векторной базе данных и ключевым словам

In [231]:
import pandas as pd
from tqdm import tqdm

# Оценка RAG-набора данных с использованием RAGAS
def evaluate_ragas_dataset(ragas_dataset):
    result = evaluate(
        ragas_dataset,
        metrics=[
            context_precision,
            faithfulness,
            answer_relevancy,
            context_recall,
            answer_correctness,
            answer_similarity
        ],
    )
    return result

# Функция для создания RAG-набора данных с использованием цепочки simple_vector_store_and_BM25
def create_ragas_dataset_with_bm25_chain(bm25_chain, eval_dataset):
    rag_dataset = []
    
    # Преобразуем Dataset в pandas DataFrame для удобной работы с данными
    eval_df = pd.DataFrame(eval_dataset)
    
    for _, row in tqdm(eval_df.iterrows()):  # Итерируем через строки DataFrame
        try:
            # Вызов цепочки через метод invoke
            result = bm25_chain.invoke({"input": row["question"], "chat_history": []})
            
            # Проверка на наличие ключей
            if not result or "answer" not in result:
                print(f"Ошибка: отсутствует ключ 'answer' для вопроса: {row['question']}")
                continue
            
            # Получаем релевантные документы (если есть)
            relevant_docs = result.get("relevant_docs", [])
            contexts = [doc.page_content for doc in relevant_docs]

            # Добавление данных в RAG-набор
            rag_dataset.append(
                {
                    "question": row["question"],
                    "answer": result["answer"],  # Ответ от цепочки
                    "contexts": contexts,  # Оставляем контексты в виде списка
                    "ground_truths": row["ground_truth"],  # Эталонные ответы
                    "reference": row["ground_truth"]  # Эталонный ответ для оценки
                }
            )
        except Exception as e:
            print(f"Ошибка обработки вопроса '{row['question']}': {e}")
    
    # Преобразуем итоговый набор в pandas DataFrame
    if rag_dataset:  # Проверяем, что набор данных не пустой
        rag_df = pd.DataFrame(rag_dataset)
        rag_eval_dataset = Dataset.from_pandas(rag_df)
        return rag_eval_dataset
    else:
        print("Ошибка: набор данных пустой.")
        return None

# Создание RAG-набора данных для оценки с использованием цепочки simple_vector_store_and_BM25 (первые 10 строк)
basic_qa_ragas_dataset_bm25 = create_ragas_dataset_with_bm25_chain(simple_vector_store_and_BM25, eval_dataset[:10])

# Проверка, что создание RAG-набора данных завершилось успешно
if basic_qa_ragas_dataset_bm25 is not None:
    # Сохранение RAG-набора данных в CSV
    basic_qa_ragas_dataset_bm25.to_csv("basic_qa_ragas_dataset_bm25.csv")

    # Оценка RAG-набора данных
    basic_qa_result_bm25 = evaluate_ragas_dataset(basic_qa_ragas_dataset_bm25)

    # Вывод результата оценки
    print(basic_qa_result_bm25)
else:
    print("Ошибка создания RAG-набора данных. Оценка не будет выполнена.")

10it [00:11,  1.12s/it]
Evaluating: 100%|██████████| 60/60 [00:21<00:00,  2.75it/s]


{'context_precision': 0.0000, 'faithfulness': 0.0000, 'answer_relevancy': 0.7049, 'context_recall': 0.0000, 'answer_correctness': 0.4183, 'answer_similarity': 0.9146}


## Оценка RAG с использованием поиска по векторной базе данных, ключевым словам и переранжированием

In [233]:
# Оценка RAG-набора данных с использованием RAGAS
def evaluate_ragas_dataset(ragas_dataset):
    result = evaluate(
        ragas_dataset,
        metrics=[
            context_precision,
            faithfulness,
            answer_relevancy,
            context_recall,
            answer_correctness,
            answer_similarity
        ],
    )
    return result

# Функция для создания RAG-набора данных с переранжированием
def create_ragas_dataset_with_reranked(reranked_chain, eval_dataset):
    rag_dataset = []
    
    # Преобразуем Dataset в pandas DataFrame
    eval_df = pd.DataFrame(eval_dataset)
    
    for _, row in tqdm(eval_df.iterrows()):  # Итерируем через строки DataFrame
        try:
            # Выполнение переранжированной цепочки для каждого вопроса
            result = reranked_chain(row["question"], chat_history=[])
            
            # Проверка на наличие ключей
            if not result or "reordered_docs" not in result or "answer" not in result:
                print(f"Ошибка: отсутствуют ключи 'reordered_docs' или 'answer' для вопроса: {row['question']}")
                continue
            
            # Проверка структуры документов
            reordered_docs = result["reordered_docs"]
            contexts = [doc.page_content for doc in reordered_docs]

            # Добавление данных в RAG-набор
            rag_dataset.append(
                {
                    "question": row["question"],
                    "answer": result["answer"],  # Ответ с использованием переранжирования
                    "contexts": contexts,  # Оставляем контексты в виде списка
                    "ground_truths": row["ground_truth"],  # Эталонные ответы
                    "reference": row["ground_truth"]  # Эталонный ответ для оценки
                }
            )
        except Exception as e:
            print(f"Ошибка обработки вопроса '{row['question']}': {e}")
    
    # Преобразуем итоговый набор в Dataset
    rag_df = pd.DataFrame(rag_dataset)
    
    rag_eval_dataset = Dataset.from_pandas(rag_df)
    
    return rag_eval_dataset

# Создание RAG-набора данных для оценки с использованием переранжированной цепочки (только первые 10 строк)
basic_qa_ragas_dataset_reranked = create_ragas_dataset_with_reranked(simple_reranked_chain, eval_dataset[:10])

# Проверка, что создание RAG-набора данных завершилось успешно
if basic_qa_ragas_dataset_reranked is not None:
    # Сохранение RAG-набора данных в CSV
    basic_qa_ragas_dataset_reranked.to_csv("basic_qa_ragas_dataset_reranked.csv")

    # Оценка RAG-набора данных
    basic_qa_result_reranked = evaluate_ragas_dataset(basic_qa_ragas_dataset_reranked)

    # Вывод результата оценки
    print(basic_qa_result_reranked)
else:
    print("Ошибка создания RAG-набора данных. Оценка не будет выполнена.")

10it [00:14,  1.49s/it]
Evaluating: 100%|██████████| 60/60 [01:46<00:00,  1.77s/it]


{'context_precision': 0.8408, 'faithfulness': 0.7750, 'answer_relevancy': 0.8986, 'context_recall': 0.7500, 'answer_correctness': 0.4878, 'answer_similarity': 0.9517}


## Оценка RAG с использованием поиска по векторной базе данных, ключевым словам и RAG-Fusion и RRF 

In [235]:
import pandas as pd
from tqdm import tqdm

# Оценка RAG-набора данных с использованием RAGAS
def evaluate_ragas_dataset(ragas_dataset):
    result = evaluate(
        ragas_dataset,
        metrics=[
            context_precision,
            faithfulness,
            answer_relevancy,
            context_recall,
            answer_correctness,
            answer_similarity
        ],
    )
    return result

# Функция для создания RAG-набора данных с использованием цепочки RAG-Fusion
def create_ragas_dataset_with_fusion_chain(fusion_chain, eval_dataset):
    rag_dataset = []
    
    # Преобразуем Dataset в pandas DataFrame для удобной работы с данными
    eval_df = pd.DataFrame(eval_dataset)
    
    for _, row in tqdm(eval_df.iterrows()):  # Итерируем через строки DataFrame
        try:
            # Вызов функции напрямую, передавая вопрос и пустую историю чата
            result = fusion_chain(row["question"], [])
            
            # Получаем релевантные документы (если есть)
            # Если результат — это строка, используем его напрямую как ответ
            answer = result if isinstance(result, str) else result.get("answer", "")
            relevant_docs = result.get("relevant_docs", []) if not isinstance(result, str) else []
            contexts = [doc.page_content for doc in relevant_docs]

            # Добавление данных в RAG-набор
            rag_dataset.append(
                {
                    "question": row["question"],
                    "answer": answer,  # Используем строковый ответ
                    "contexts": contexts,  # Оставляем контексты в виде списка
                    "ground_truths": row["ground_truth"],  # Эталонные ответы
                    "reference": row["ground_truth"]  # Эталонный ответ для оценки
                }
            )
        except Exception as e:
            print(f"Ошибка обработки вопроса '{row['question']}': {e}")
    
    # Преобразуем итоговый набор в pandas DataFrame
    if rag_dataset:  # Проверяем, что набор данных не пустой
        rag_df = pd.DataFrame(rag_dataset)
        rag_eval_dataset = Dataset.from_pandas(rag_df)
        return rag_eval_dataset
    else:
        print("Ошибка: набор данных пустой.")
        return None

# Создание RAG-набора данных для оценки с использованием цепочки RAG-Fusion (первые 10 строк)
basic_qa_ragas_dataset_fusion = create_ragas_dataset_with_fusion_chain(process_query, eval_dataset[:10])

# Проверка, что создание RAG-набора данных завершилось успешно
if basic_qa_ragas_dataset_fusion is not None:
    # Сохранение RAG-набора данных в CSV
    basic_qa_ragas_dataset_fusion.to_csv("basic_qa_ragas_dataset_fusion.csv")

    # Оценка RAG-набора данных
    basic_qa_result_fusion_RRF= evaluate_ragas_dataset(basic_qa_ragas_dataset_fusion)

    # Вывод результата оценки
    print(basic_qa_result_fusion_RRF)
else:
    print("Ошибка создания RAG-набора данных. Оценка не будет выполнена.")

10it [00:40,  4.02s/it]
Evaluating: 100%|██████████| 60/60 [00:19<00:00,  3.11it/s]


{'context_precision': 0.0000, 'faithfulness': 0.3500, 'answer_relevancy': 0.7975, 'context_recall': 0.0000, 'answer_correctness': 0.4891, 'answer_similarity': 0.9485}


	•	RRF — это мощный метод, который использует ранжирование результатов из разных источников для слияния в единый список. Однако, если результирующие документы, полученные через ensemble_retriever, имеют высокую степень схожести или одинаковые данные (из-за пересечения в запросах), RRF может вносить дополнительный “шум” в ранжирование и сделать его менее четким.
	•	Генерация нескольких запросов и их последующий анализ не всегда гарантирует улучшение, особенно если поисковые движки, используемые ensemble_retriever, имеют сходные индексы. Это может приводить к дублированию информации или незначительным различиям, что не увеличивает разнообразие результатов.

In [239]:
# Создание списка данных и имен
data = [
    ('basic_qa_vector_store', basic_qa_vector_store),
    ('basic_qa_result_bm25', basic_qa_result_bm25),
    ('basic_qa_result_reranked', basic_qa_result_reranked),
    ('basic_qa_result_fusion_RRF', basic_qa_result_fusion_RRF),
]

# Создание DataFrame из каждого словаря с добавлением имени
rows = []
for name, values in data:
    row = {'name': name}
    row.update(values)
    rows.append(row)

# Создание финального DataFrame
df = pd.DataFrame(rows)

# Печать таблицы
df

,name,context_precision,faithfulness,answer_relevancy,context_recall,answer_correctness,answer_similarity
0,basic_qa_vector_store,0.000000,0.300,0.702250,0.00,0.405394,0.909810
1,basic_qa_result_bm25,0.000000,0.000,0.704887,0.00,0.418296,0.914565
2,basic_qa_result_reranked,0.840822,0.775,0.898564,0.75,0.487764,0.951729
3,basic_qa_result_fusion_RRF,0.000000,0.350,0.797542,0.00,0.489093,0.948525


In [263]:
import pandas as pd

# Данные таблицы
data = {
    "name": ["basic_qa_vector_store", "basic_qa_result_bm25", "basic_qa_result_reranked", "basic_qa_result_fusion_RRF"],
    "context_precision": [0.0, 0.0, 0.840822, 0.0],
    "faithfulness": [0.3, 0.0, 0.775, 0.35],
    "answer_relevancy": [0.70225, 0.704887, 0.898564, 0.797542],
    "context_recall": [0.0, 0.0, 0.75, 0.0],
    "answer_correctness": [0.405394, 0.418296, 0.487764, 0.489093],
    "answer_similarity": [0.90981, 0.914565, 0.951729, 0.948525]
}

# Создание DataFrame
df = pd.DataFrame(data)

# Сохранение таблицы в CSV файл
csv_path = "/Users/sergey/Desktop/RAG_Project_FULL/RAG_intelion/Intelion_books/retrieval_chain_results.csv"
df.to_csv(csv_path, index=False)

csv_path

'/Users/sergey/Desktop/RAG_Project_FULL/RAG_intelion/Intelion_books/retrieval_chain_results.csv'